# Imports and Setups

In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 6.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


Lendo o Arquivo

In [1]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [2]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()


In [3]:
num_gen = 5
perc = 5
i=4

In [4]:
arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(i)+'.csv'
base_cru_1p = pd.read_csv(arq, encoding='utf-8', sep=';', decimal = ',')

In [5]:
base_cru_1p.head()

,texto,label
0,@user LOL. What’s failing miserably is this 11...,1
1,#cincinatti well these shootings will keep ha...,1
2,#TheresaMay we need a #PeoplesVote @user,0
3,@user Liberals don't care anymore about democr...,0
4,@user I don't believe she's even getting death...,1


In [6]:
def cria_prompt(ordem,synt):
  prompt = ordem

  prompt += 'Tweet: '+ synt+'\nResult: '
  return prompt

In [7]:
inicio = """Rewrite the following tweets 5 times to generate 5 different paraphrases  from original tweet.
Return in Python list like this ["paraphrase1","paraphrase2","paraphrase3","paraphrase4","paraphrase5"] as example.

Tweet:What are the best places to see in New York?
Result:["What are some must-see places in New York?","Can you suggest some must-see spots in New York?","Where should one go to experience the best NYC has to offer?","Which places should I visit in New York?","What are the top destinations to explore in New York?"]
###
"""
base_cru_1p['prompt'] = base_cru_1p.progress_apply(lambda x: cria_prompt(inicio, x.texto), axis=1)
base_cru_1p.head()

100%|██████████| 170/170 [00:00<00:00, 8099.60it/s]


,texto,label,prompt
0,@user LOL. What’s failing miserably is this 11...,1,Rewrite the following tweets 5 times to genera...
1,#cincinatti well these shootings will keep ha...,1,Rewrite the following tweets 5 times to genera...
2,#TheresaMay we need a #PeoplesVote @user,0,Rewrite the following tweets 5 times to genera...
3,@user Liberals don't care anymore about democr...,0,Rewrite the following tweets 5 times to genera...
4,@user I don't believe she's even getting death...,1,Rewrite the following tweets 5 times to genera...


In [8]:
base_cru_1p.texto.iloc[90]

'&gt;Literally runs through the wall. There was now a hole.  Master! You are okay?'

In [9]:
print(base_cru_1p.prompt.iloc[90])

Rewrite the following tweets 5 times to generate 5 different paraphrases  from original tweet.
Return in Python list like this ["paraphrase1","paraphrase2","paraphrase3","paraphrase4","paraphrase5"] as example.

Tweet:What are the best places to see in New York?
Result:["What are some must-see places in New York?","Can you suggest some must-see spots in New York?","Where should one go to experience the best NYC has to offer?","Which places should I visit in New York?","What are the top destinations to explore in New York?"]
###
Tweet: &gt;Literally runs through the wall. There was now a hole.  Master! You are okay?
Result: 


In [10]:
n_labels = base_cru_1p.label.nunique()
n_labels

2

In [11]:
! huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
Cannot authenticate through git-cr

In [12]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "meta-llama/Llama-2-7b-chat-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model_4bit = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
prompt = """Rewrite the following tweets 5 times to generate 5 different paraphrases  from original tweet.
Return in Python list like this ["paraphrase1","paraphrase2","paraphrase3","paraphrase4","paraphrase5"] as example.

Tweet:What are the best places to see in New York?
Result:["What are some must-see places in New York?","Can you suggest some must-see spots in New York?","Where should one go to experience the best NYC has to offer?","Which places should I visit in New York?","What are the top destinations to explore in New York?"]
###
Tweet: The Quarterback' wrecks me every time..
Result: """

In [14]:
def gera_text(modelo, tokens, dev, ppt, temperatura, max_tokens):
  inputs = tokens(ppt, return_tensors="pt").to(dev)
  outputs = modelo.generate(**inputs, max_new_tokens=max_tokens, temperature=temperatura, do_sample =True)
  result = tokenizer.decode(outputs[0], skip_special_tokens=True)
  return result

In [15]:
device = "cuda:0"
print(gera_text(model_4bit, tokenizer, device, prompt, 0.1, 400))

Rewrite the following tweets 5 times to generate 5 different paraphrases  from original tweet.
Return in Python list like this ["paraphrase1","paraphrase2","paraphrase3","paraphrase4","paraphrase5"] as example.

Tweet:What are the best places to see in New York?
Result:["What are some must-see places in New York?","Can you suggest some must-see spots in New York?","Where should one go to experience the best NYC has to offer?","Which places should I visit in New York?","What are the top destinations to explore in New York?"]
###
Tweet: The Quarterback' wrecks me every time..
Result:  ["The Quarterback's impact never fails to move me...", "The Quarterback's brilliance always leaves me in awe...", "The Quarterback's talent is unmatched and always brings me to tears...", "The Quarterback's greatness never ceases to amaze me...", "The Quarterback's talent is unparalleled and always leaves me breathless..."]






















































































In [16]:
base_cru_1p['generated'] = base_cru_1p['prompt'].progress_apply(lambda x: gera_text(model_4bit, tokenizer, device, x, 0.1, 400))
base_cru_1p.head()

100%|██████████| 170/170 [1:48:44<00:00, 38.38s/it]


,texto,label,prompt,generated
0,@user LOL. What’s failing miserably is this 11...,1,Rewrite the following tweets 5 times to genera...,Rewrite the following tweets 5 times to genera...
1,#cincinatti well these shootings will keep ha...,1,Rewrite the following tweets 5 times to genera...,Rewrite the following tweets 5 times to genera...
2,#TheresaMay we need a #PeoplesVote @user,0,Rewrite the following tweets 5 times to genera...,Rewrite the following tweets 5 times to genera...
3,@user Liberals don't care anymore about democr...,0,Rewrite the following tweets 5 times to genera...,Rewrite the following tweets 5 times to genera...
4,@user I don't believe she's even getting death...,1,Rewrite the following tweets 5 times to genera...,Rewrite the following tweets 5 times to genera...


In [17]:
base_cru_1p.head()

,texto,label,prompt,generated
0,@user LOL. What’s failing miserably is this 11...,1,Rewrite the following tweets 5 times to genera...,Rewrite the following tweets 5 times to genera...
1,#cincinatti well these shootings will keep ha...,1,Rewrite the following tweets 5 times to genera...,Rewrite the following tweets 5 times to genera...
2,#TheresaMay we need a #PeoplesVote @user,0,Rewrite the following tweets 5 times to genera...,Rewrite the following tweets 5 times to genera...
3,@user Liberals don't care anymore about democr...,0,Rewrite the following tweets 5 times to genera...,Rewrite the following tweets 5 times to genera...
4,@user I don't believe she's even getting death...,1,Rewrite the following tweets 5 times to genera...,Rewrite the following tweets 5 times to genera...


In [18]:
import ast
def transforme_list(x):
  try:
    resp = ast.literal_eval(x.split("Result:")[2].replace("\n", "").strip())
  except:
    resp = []
  return resp

In [19]:
print(base_cru_1p.generated.iloc[4])

Rewrite the following tweets 5 times to generate 5 different paraphrases  from original tweet.
Return in Python list like this ["paraphrase1","paraphrase2","paraphrase3","paraphrase4","paraphrase5"] as example.

Tweet:What are the best places to see in New York?
Result:["What are some must-see places in New York?","Can you suggest some must-see spots in New York?","Where should one go to experience the best NYC has to offer?","Which places should I visit in New York?","What are the top destinations to explore in New York?"]
###
Tweet: @user I don't believe she's even getting death threats. Conservatives don't roll that way. Plus her &amp; her attorneys bankroll is Soros
Result:  ["@user I don't think she's receiving death threats. Conservatives don't typically engage in that type of behavior. Additionally, her and her legal team's financial backer is George Soros.","@user I don't believe she's getting death threats. Conservatives usually don't resort to such extreme measures. Furthermo

In [20]:
base_cru_1p.generated.iloc[0].split("Result:")[2].replace("\n", "").strip()

'["What\'s hilarious is this 11th hour ambush by liberals. Every minute #ChristinaBlaseyFord refuses to commit to appearing, she and people like you lose credibility. I love it!! 😂😂 #tcot #p2", "The liberals\' 11th hour ambush is failing miserably. Every minute #ChristinaBlaseyFord refuses to commit to appearing, she and people like you lose credibility. I love it!! 😂😂 #tcot #p2", "It\'s hilarious to see the liberals\' 11th hour ambush failing. Every minute #ChristinaBlaseyFord refuses to commit to appearing, she and people like you lose credibility. I love it!! 😂😂 #tcot #p2", "The 11th hour ambush by liberals is backfiring. Every minute #ChristinaBlaseyFord refuses to commit to appearing, she and people like you lose credibility. I love it!! 😂😂 #tcot #p2", "Liberals\' 11th hour ambush is a joke. Every minute #ChristinaBlaseyFord refuses to commit to appearing, she and people like you lose credibility. I love it!! 😂😂 #tcot #p2"]'

In [21]:
print(ast.literal_eval(base_cru_1p.generated.iloc[3].split("Result:")[2].replace("\n", "")))

SyntaxError: ignored

In [22]:
base_cru_1p['generated'] = base_cru_1p['generated'].progress_apply(lambda x: transforme_list(x))

100%|██████████| 170/170 [00:00<00:00, 13040.80it/s]


In [23]:
base_cru_1p.head()

,texto,label,prompt,generated
0,@user LOL. What’s failing miserably is this 11...,1,Rewrite the following tweets 5 times to genera...,[What's hilarious is this 11th hour ambush by ...
1,#cincinatti well these shootings will keep ha...,1,Rewrite the following tweets 5 times to genera...,[#Cincinnati shootings will continue until US ...
2,#TheresaMay we need a #PeoplesVote @user,0,Rewrite the following tweets 5 times to genera...,[#TheresaMay should we have a #PeoplesVote? @u...
3,@user Liberals don't care anymore about democr...,0,Rewrite the following tweets 5 times to genera...,[]
4,@user I don't believe she's even getting death...,1,Rewrite the following tweets 5 times to genera...,[@user I don't think she's receiving death thr...


In [24]:
export = base_cru_1p.copy()
export = export.explode('generated').reset_index(drop=True)


In [25]:
export.head()

,texto,label,prompt,generated
0,@user LOL. What’s failing miserably is this 11...,1,Rewrite the following tweets 5 times to genera...,What's hilarious is this 11th hour ambush by l...
1,@user LOL. What’s failing miserably is this 11...,1,Rewrite the following tweets 5 times to genera...,The liberals' 11th hour ambush is failing mise...
2,@user LOL. What’s failing miserably is this 11...,1,Rewrite the following tweets 5 times to genera...,It's hilarious to see the liberals' 11th hour ...
3,@user LOL. What’s failing miserably is this 11...,1,Rewrite the following tweets 5 times to genera...,The 11th hour ambush by liberals is backfiring...
4,@user LOL. What’s failing miserably is this 11...,1,Rewrite the following tweets 5 times to genera...,Liberals' 11th hour ambush is a joke. Every mi...


In [26]:
export.shape

(770, 4)

In [27]:
base_cru_1p.shape

(170, 4)

In [28]:
export.shape[0]/base_cru_1p.shape[0]

4.529411764705882

In [29]:
arquivo = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/gptparaphrase/base_offensive_'+str(perc)+'_gptparaphrase'+str(i)+'.pkl'
export[['texto', 'label', 'generated']].to_pickle(arquivo)